<a href="https://colab.research.google.com/github/don05050505/don05050505/blob/main/MnistClsf2HL0825.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.initializers import RandomNormal

class MnistClassificationTwoHiddenLayerModel(tf.keras.Model):
    def __init__(self, numOf1HiddenNode, numOf2HiddenNode, numOfOutput):
        super(MnistClassificationTwoHiddenLayerModel, self).__init__()

        self.hiddenLayer1 = tf.keras.layers.Dense(numOf1HiddenNode, activation='relu')
        self.hiddenLayer2 = tf.keras.layers.Dense(numOf2HiddenNode, activation='relu')
        self.outPut = tf.keras.layers.Dense(numOfOutput, activation='linear')  # Linear activation for logits

    def call(self, x): #forward
        y1 = self.hiddenLayer1(x)
        y2 = self.hiddenLayer2(y1)
        y3 = self.outPut(y2)
        return y3

@tf.function
def cross_entropy(ypred, y):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=ypred, labels=y))

@tf.function
def backward(model, optimizer, x, y):
    with tf.GradientTape() as gt:
        yPred = model(x)
        loss = cross_entropy(yPred, y)
    gradz = gt.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradz, model.trainable_variables))

@tf.function
def accuracy(yPred, y):
    correction = tf.equal(tf.argmax(yPred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(correction, tf.float32))
    return acc

# Load and preprocess the MNIST dataset
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0
train_images = train_images.reshape(train_images.shape[0], -1)
test_images = test_images.reshape(test_images.shape[0], -1)
train_labels = tf.one_hot(train_labels, depth=10)
test_labels = tf.one_hot(test_labels, depth=10)
batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(buffer_size=10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(batch_size)

# Define your model
model = MnistClassificationTwoHiddenLayerModel(numOf1HiddenNode=128, numOf2HiddenNode=64, numOfOutput=10)

# Define an optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for x_batch, y_batch in train_dataset:
        backward(model, optimizer, x_batch, y_batch)

    # Calculate and display accuracy after each epoch
    acc_sum = 0
    num_batches = 0
    for x_test_batch, y_test_batch in test_dataset:
        y_test_pred = model(x_test_batch)
        acc = accuracy(y_test_pred, y_test_batch)
        acc_sum += acc
        num_batches += 1
    avg_acc = acc_sum / num_batches
    print(f"Epoch {epoch+1}, Average Test Accuracy: {avg_acc}")
